In [1]:
import os
%pwd

'c:\\Users\\RafaelRaiserQSOFT\\Stellar-Classification\\research'

In [2]:
os.chdir('../')
%pwd


'c:\\Users\\RafaelRaiserQSOFT\\Stellar-Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path


In [4]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
from src import logger
from src.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2024-02-15 10:54:23,178: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-15 10:54:23,180: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-15 10:54:23,182: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-15 10:54:23,186: INFO: common: created directory at: artifacts]
[2024-02-15 10:54:23,188: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-15 10:54:24,257: INFO: 1567088110: artifacts/data_ingestion/data.csv download! with following info: 
Connection: close
Content-Length: 16755863
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: text/plain; charset=utf-8
ETag: "0c7e654307d4120d30bcc11ae5aac6736522fda31a07b12fb874b709895d2dfe"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 789C:202207:100F8:126A8:65CE1789
Accept-Ranges: bytes
Da